# Advanced analysis of estimates

👋 Hello, this sample will showcase different experiments that can be created on
top of Azure Quantum Resource Estimator.  The sample will re-use some of the
implementations from the _Estimates with Q#_ notebook.  Please refer to that
notebook for more details on the setup and algorithm implementation.

## Setup

Let's connect to the Azure Quantum workspace and select the Azure Quantum
Resource Estimator as target.  We are also importing the
`Microsoft.Quantum.Numerics` package that we will require for our example
algorithm.

In [ ]:
from ipywidgets import IntProgress, Layout # To show interactive progress while job submission
from IPython.display import display, HTML  # To display HTML inside Jupyter notebooks
import time                                # To sleep while polling for job completion
import numpy as np                         # To store experimental data from job results
from matplotlib import pyplot as plt       # To plot experimental results
from matplotlib.colors import hsv_to_rgb   # To automatically find colors for plots

import qsharp

In [ ]:

import qsharp.azure
targets = qsharp.azure.connect(
    resourceId="",
    location="")

In [ ]:
qsharp.packages.add("Microsoft.Quantum.Numerics")
qsharp.azure.target("microsoft.estimator")

## Implementing the algorithm

As running example algorithm we are creating a multiplier using the [MultiplyI](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.arithmetic.multiplyi) operation.  We can configure the size of the multiplier with a bitwidth parameter. The operation will have two input registers with that bitwidth, and one output register with the size of twice the bitwidth.

In [ ]:
%%qsharp

open Microsoft.Quantum.Arithmetic;

operation EstimateMultiplication(bitwidth : Int) : Unit {
    use factor1 = Qubit[bitwidth];
    use factor2 = Qubit[bitwidth];
    use product = Qubit[2 * bitwidth];

    MultiplyI(LittleEndian(factor1), LittleEndian(factor2), LittleEndian(product));
}

## Setting up and running the experiments

Next, we are setting up some experiments. Here, we are using two of the six
pre-defined qubit parameter models, and one customized model based on the model
`qubit_gate_nds_e3`, in which we change the error rates to $10^{-3.5}$. In your
own experiments, you can change the number of items, and also the parameters.
You may use other pre-defined models or define custom models. You can find more
information about the input parameters in the _Getting Started with Azure
Quantum Resource Estimation_ notebook.

Further, we are choosing a list of input parameters to our algorithm, in this
case bitwidths that are powers-of-2 ranging from 8 to 64.

In [ ]:
input_params = {
    "Gate-based ns, 10⁻³": {"qubitParams": {"name": "qubit_gate_ns_e3"}},
    "Gate-based ns, 10⁻³ᐧ⁵": {"qubitParams": {"name": "qubit_gate_ns_e3", "oneQubitMeasurementErrorRate": 0.00032, "oneQubitGateErrorRate": 0.00032, "twoQubitGateErrorRate": 0.00032, "tGateErrorRate": 0.00032}},
    "Gate-based ns, 10⁻⁴": {"qubitParams": {"name": "qubit_gate_ns_e4"}}
}

bitwidths = [8, 16, 32, 64]

# We also store the names of the experiments; if you like to force some order
# you can explicitly initialize the list with names from the `input_params`
# dictionary.
names = list(input_params.keys())

We are now submitting resource estimation jobs for all combinations of job
parameters and input arguments.

Since the Resource Estimator does not support input parameters for operations,
we are creating a wrapper operation on the fly by inserting the bitwidth
directly into the source code and then compiling it using `qsharp.compile`.
This is a generic method that you can use to generate operations for resource
estimation from Python.

We then submit this wrapper operation for each experiment configuration using
`qsharp.azure.submit`.  This will return a job object from which we extract the
Job ID. and store it in the `jobs` dictionary.  Note that loop will not wait for
jobs to be finished.

In [ ]:
# This initializes a `jobs` dictionary with the same keys as `input_params` and
# empty arrays as values
jobs = {name: [] for name in names}

progress_bar = IntProgress(min=0, max=len(input_params) * len(bitwidths) - 1, style={'description_width': 'initial'}, layout=Layout(width='75%'))
display(progress_bar)

for bitwidth in bitwidths:
    callable = qsharp.compile(f"""operation EstimateMultiplication{bitwidth}() : Unit {{ EstimateMultiplication({bitwidth}); }}""");
    print(callable)

    for name, params in input_params.items():
        progress_bar.description = f"{bitwidth}: {name}"

        result = qsharp.azure.submit(callable, jobParams=params)
        jobs[name].append(result.id)
        progress_bar.value += 1

The next code block is commented out.  But it shows some ways how to avoid
re-submitting the same job.  For example,

* after running the jobs above you can print out the jobs using the `print`
  command in comments and then paste it into the cell.  Like this you can easily
  access the job IDs in future sessions without needing to re-submit jobs.
* after running jobs in some other notebook and collecting them here, you can
  paste the job IDs that you can access from the _Job management_ page in your
  _Azure Quantum Workspace_.

In [ ]:
# # Use the following line to print all job IDs and then update them in the bottom of the cell
# print(f"jobs = {jobs}")

# # Update and uncomment this line if you want to re-use pre-computed jobs in the future.
# # These job ids are not complete and are just printed to provide an idea of what to expect from the output.
# # See the line above on how to generate this line
# jobs = {'Gate-based ns, 10⁻³': ['fdd354d9-...', ...], 'Gate-based ns, 10⁻³ᐧ⁵': ['453f7039-...', ...], 'Gate-based ns, 10⁻⁴': ['cf273c84-...', ...]}

All jobs have been submitted now.  But they may have not been finished.  The
next code cell is extracting the resource estimation results from each job.  To
do that, it will first wait for a job to have succeeded, whenever it is still in
a waiting or executing state.  All results are saved to a `results` dictionary,
that has an array for each experiment name that has all corresponding results
sorted by bitwidth.

In [ ]:
# This initializes a `results` dictionary with the same keys as `input_params`
# and empty arrays as values
results = {name: [] for name in names}

progress_bar = IntProgress(min=0, max=len(input_params) * len(bitwidths) - 1, style={'description_width': 'initial'}, layout=Layout(width='75%'))
display(progress_bar)

for name, job_ids in jobs.items():
    for job_id in job_ids:
        progress_bar.description = job_id

        # Wait until a job has succeeded or failed
        while True:
            status = qsharp.azure.status(job_id)
            if status.status in ["Waiting", "Executing"]:
                time.sleep(1) # Waits one second
            elif status.status == "Succeeded":
                break
            else:
                raise Exception(f"{status.status} job {job_id} in {name}")

        result = qsharp.azure.output(job_id)
        results[name].append(result)
        progress_bar.value += 1

## Plotting the experimental results

Now that we have all results, we extract some data from it.  We extract the
number of physical qubits, the total runtime in nanoseconds, and the QEC code
distance for the logical qubits.  In addition to the total number of physical
qubits, we are also extracting their breakdown into number of physical qubits
for executing the algorithm and the number of physical qubits required for the T
factories that produce the required T states.

In [ ]:
names = list(input_params.keys())

qubits = np.zeros((len(names), len(bitwidths), 3))
runtime = np.zeros((len(names), len(bitwidths)))
distances = np.zeros((len(names), len(bitwidths)))

for bitwidth_index, bitwidth in enumerate(bitwidths):
    for name_index, name in enumerate(names):
        data = results[names[name_index]][bitwidth_index]

        qubits[(name_index, bitwidth_index, 0)] = data['physicalCounts']['physicalQubits']
        qubits[(name_index, bitwidth_index, 1)] = data['physicalCounts']['breakdown']['physicalQubitsForAlgorithm']
        qubits[(name_index, bitwidth_index, 2)] = data['physicalCounts']['breakdown']['physicalQubitsForTfactories']

        runtime[(name_index, bitwidth_index)] = data['physicalCounts']['runtime']

        distances[(name_index, bitwidth_index)] = data['logicalQubit']['codeDistance']

Finally, we are using [Matplotlib](https://matplotlib.org/) to plot the number
of physical qubits and the runtime as bar plots, and the QEC code distances as a
scatter plot.  For the physical qubits, we are showing the partition into qubits
required for the algorithm and qubits required for the T factories.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(22, 6))

num_experiments = len(names)                         # Extract number of experiments form names (can be made smaller)
xs = np.arange(0, len(bitwidths))                    # Map bitwidths to numeric indexes for plotting
full_width = .8                                      # Total width of all bars (should be smaller than 1)
width = full_width / num_experiments                 # Fractional width of a single bar
xs_left = xs - (((num_experiments - 1) * width) / 2) # Starting x-coordinate for bars

# Split axes into qubit and runtime plots
ax_qubits, ax_runtime, ax_code_distance = axs

# Plot physical qubits
for i in range(num_experiments):
    ax_qubits.bar(xs_left + i * width, qubits[i,:,1], width, label=f"{names[i]} (Alg.)", color=hsv_to_rgb((i / num_experiments, 1.0, .8)))
    ax_qubits.bar(xs_left + i * width, qubits[i,:,2], width, bottom=qubits[i,:,1], label=f"{names[i]} (T fac.)", color=hsv_to_rgb((i / num_experiments, 0.3, .8)))
ax_qubits.set_title("#Physical qubits")
ax_qubits.set_xlabel("Bitwidth")
ax_qubits.set_xticks(xs)
ax_qubits.set_xticklabels(bitwidths)
ax_qubits.legend()

# Plot runtime
for i in range(num_experiments):
    ax_runtime.bar(xs_left + i * width, np.array(runtime[i,:]) / 1e6, width, label=names[i], color=hsv_to_rgb((i / num_experiments, 1.0, .8)))
ax_runtime.set_title("Runtime (ms)")
ax_runtime.set_xlabel("Bitwidth")
ax_runtime.set_xticks(xs)
ax_runtime.set_xticklabels(bitwidths)
ax_runtime.legend()

# Plot code distances
for i in range(num_experiments):
    ax_code_distance.scatter(xs, distances[i,:], label=names[i], marker='*', color=hsv_to_rgb((i / num_experiments, 1.0, 0.8)))
ax_code_distance.set_title("QEC code distance")
ax_code_distance.set_xlabel("Bitwidth")
ax_code_distance.set_xticks(xs)
ax_code_distance.set_xticklabels(bitwidths)
ax_code_distance.legend()

fig.suptitle("Resource estimates for multiplication")
plt.show()

## Showing resource estimates in custom tables

You have probably already seen the resource estimation table that you can get
for a single result.  But did you know that all the data required to output the
table is also part of the resource estimation results?  You can access all that
data using the `'reportData'` key from the results dictionary.  You can use this
data to create your own tables.  In the next code block we show how to create a
side-to-side comparison table for the _T-factory parameters_ from the resource
estimation results for all input parameters and a fixed bitwidth.

In [ ]:
bitwidth = 16 # Choose one of the bitwidths here
bitwidth_index = bitwidths.index(bitwidth)

# Get all results from all input parameters for given bitwidth
data = [results[name][bitwidth_index] for name in names]

# From each result get the group that contains data about "T-factory parameters"
groups = [group for result in data for group in result['reportData']['groups'] if group['title'] == "T factory parameters"]

html = "<table><thead><tr><th></th>"

# Produce a table header using the experiment names
for name in names:
    html += f"<td>{name}</th>"

html += "</tr></thead><tbody>"

# Iterate through all entries (we extract the count from the first group, and then iterate through all of them)
for entry_index in range(len(groups[0]['entries'])):
    # Extract the entry label from the first group
    html += f"""<tr><td style="text-align: left; font-weight: bold">{groups[0]['entries'][entry_index]['label']}</td>"""

    # Iterate through all experiments
    for group_index in range(len(groups)):
        # The 'path' variable in the entry is a '/'-separated path to access the
        # result dictionary. So we start from the result dictionary of the
        # current experiment and then access the field based on the path part.
        # Eventually we obtain the final value.
        value = data[group_index]
        for key in groups[group_index]['entries'][entry_index]['path'].split("/"):
            value = value[key]
        html += f"<td>{value}</td>"
    html += "</tr>"

html += "</tbody></table>"

HTML(html)

## Next steps

We hope you got some ideas and inspirations for your own resource estimation
experiments.  Feel free to use this notebook as a starting point for your own
algorithm investigations.  To get more familiar with resource estimation
experiments, here are some suggestions to try out in this notebook:

* Add a fourth plot to show some statistics about a single T factory, e.g., its
  number of qubits.
* Add a new plot series to show logical resource estimates.
* Change the algorithm to create an $n$-ary multiplier (with a variable number
  of input arguments) for either a fixed or customizable bitwidth.
* Create a side-by-side comparison table for the logical qubit parameters.